In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import word_tokenize
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import json

# read date

In [57]:
with open('train.json', encoding='UTF-8-SIG') as f:
    raw_train = json.load(f)

In [58]:
raw_train[1]

{'text': 'Медики рассказали о состоянии пострадавшего мужчины, на которого было совершено нападение возле отделения банка по Тимирязева. Как прокомментировали Tengrinews.kz в пресс-службе Управления здравоохранения Алматы, с места происшествия в службу скорой помощи обратились двое человек. \n\n«Одному из них на месте была оказана медицинская помощь. От госпитализации он отказался. Второй пациент был доставлен в больницу скорой неотложной помощи (БСНП) с сотрясением головного мозга, ушибленной раной головы. Состояние на данный момент оценивается ближе к удовлетворительному. Пока он проходит обследование в больнице», — сообщили в Управлении здравоохранения Алматы.  \n\nНапомним, в Алматы на пересечении улиц Тимирязева и Маркова возле БЦ «Алатау Гранд» произошла стрельба, ориентировочно в обеденное время. В здании расположены отделения банков «ВТБ» и «Сбербанк». \n\nВ настоящее время полицейские разыскивают подозреваемых в стрельбе. По факту нападения в местном управлении внутренних дел 

## build tfidf language model

In [59]:
def ru_token(string):
    """russian tokenize based on nltk.word_tokenize. only russian letter remaind."""
    return [i for i in word_tokenize(string) if re.match(r'[\u0400-\u04ffа́]+$', i)]

In [60]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\totmi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [61]:
params = {}
params['tokenizer'] = ru_token
params['stop_words'] = nltk.corpus.stopwords.words('russian')
params['ngram_range'] = (1, 3)
params['min_df'] = 3

In [62]:
tfidf  = TfidfVectorizer(**params)

In [63]:
tfidf.fit([i['text'] for i in raw_train])

C:\Users\totmi\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(min_df=3, ngram_range=(1, 3),
                stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с',
                            'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его',
                            'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы',
                            'по', 'только', 'ее', 'мне', ...],
                tokenizer=<function ru_token at 0x000001EFC8B9C670>)

## train Validation set split

In [64]:
train = {}
val = {}
tmp = defaultdict(list)
for e in raw_train:
    tmp[e['sentiment']].append(e['text'])
for l in tmp:
    train[l], val[l] = train_test_split(tmp[l], test_size=0.25, random_state=42)

## upsampling align for balance

In [65]:
def upsampling_align(some_dict, random_state=2018):
    rand = np.random.RandomState(random_state)
    upper = max([len(some_dict[l]) for l in some_dict])
    print('upper bound: {}'.format(upper))
    tmp = {}
    for l in some_dict:
        if len(some_dict[l]) < upper:
            repeat_time = int(upper/len(some_dict[l]))
            remainder = upper % len(some_dict[l])
            _tmp = some_dict[l].copy()
            rand.shuffle(_tmp)
            tmp[l] = some_dict[l] * repeat_time + _tmp[:remainder]
            rand.shuffle(tmp[l])
        else:
            tmp[l] = some_dict[l]
    return tmp

In [66]:
btrain = upsampling_align(train)

upper bound: 3025


## softmax regression model training

In [67]:
m_params = {}
m_params['solver'] = 'lbfgs'
m_params['multi_class'] = 'multinomial'

In [68]:
softmax = LogisticRegression(**m_params)

In [69]:
train_x = [j for i in sorted(btrain.keys()) for j in btrain[i]]
train_y = [i for i in sorted(btrain.keys()) for j in btrain[i]]
softmax.fit(tfidf.transform(train_x), train_y)

C:\Users\totmi\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='multinomial')

## evaluate the softmax regression model

### accuracy

In [70]:
test_x = [j for i in sorted(val.keys()) for j in val[i]]
true = [i for i in sorted(val.keys()) for j in val[i]]

In [71]:
pred = softmax.predict(tfidf.transform(test_x))

In [72]:
accuracy_score(true, pred)

0.7155297532656023

### macro recall

In [73]:
lab = LabelEncoder()
c_true = lab.fit_transform(true)
c_pred = lab.transform(pred)
print(classification_report(c_true, c_pred, target_names=lab.classes_, digits=5))

              precision    recall  f1-score   support

    negative    0.66227   0.69916   0.68022       359
     neutral    0.75510   0.69673   0.72474      1009
    positive    0.69353   0.75107   0.72115       699

    accuracy                        0.71553      2067
   macro avg    0.70363   0.71566   0.70870      2067
weighted avg    0.71816   0.71553   0.71580      2067



### balance score

In [74]:
bval = upsampling_align(val)

upper bound: 1009


In [75]:
b_test_x = [j for i in sorted(bval.keys()) for j in bval[i]]
b_true = [i for i in sorted(bval.keys()) for j in bval[i]]
b_pred = softmax.predict(tfidf.transform(b_test_x))
lab = LabelEncoder()
c_true = lab.fit_transform(b_true)
c_pred = lab.transform(b_pred)
print(classification_report(c_true, c_pred, target_names=lab.classes_, digits=5))

              precision    recall  f1-score   support

    negative    0.83790   0.69673   0.76082      1009
     neutral    0.61237   0.69673   0.65183      1009
    positive    0.72404   0.74628   0.73499      1009

    accuracy                        0.71325      3027
   macro avg    0.72477   0.71325   0.71588      3027
weighted avg    0.72477   0.71325   0.71588      3027



## predict

In [76]:
with open('TrainOnlySentenceJson.json', encoding='UTF-8-SIG') as f:
    raw_test = json.load(f)

In [77]:
sub_pred = softmax.predict(tfidf.transform([i['text'] for i in raw_test]))
sub_df = pd.DataFrame()
sub_df['id'] =  [i['id'] for i in raw_test]
sub_df['text'] =  [g['text'] for g in raw_test]
sub_df['sentiment'] = sub_pred

In [78]:
sub_df.head()

,id,text,sentiment
0,1,Один из создателей сервиса шестисекундных виде...,positive
1,2,Об этом сообщает портал TMZ со ссылкой на исто...,positive
2,3,"По информации издания, утром в воскресенье, 16...",neutral
3,4,После этого сотрудники правоохранительных орга...,negative
4,5,В комнате были найдены также следы наркотиков ...,neutral


In [79]:
sub_df.to_csv('SentimentAnalysis.csv', index=False)